In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Run this in its own cell
print("--- Installing/Upgrading Qiskit packages ---")
!pip install --upgrade qiskit qiskit-aer qiskit-experiments qiskit-ibm-runtime matplotlib
print("--- Installation Complete ---")


# Step 1: Generate Training Data & Classical Baseline

In [ ]:
# --- Installation ---
# Run this once at the top of your notebook.
# You may need to restart the runtime after installation.
!pip install --upgrade --quiet qiskit qiskit-aer qiskit-experiments qiskit-ibm-runtime matplotlib ipython

# --- Python Code ---
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Qiskit Core
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# Qiskit Aer for simulation
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Qiskit Experiments for Tomography
from qiskit_experiments.library import StateTomography

# Fake-backend (for realistic noise model)
from qiskit_ibm_runtime.fake_provider import FakeVigoV2

# Qiskit visualization utilities
from qiskit.visualization import plot_state_city, plot_bloch_multivector

def main():
    """
    Runs a 1-qubit state tomography experiment to establish a baseline.
    Generates noisy 'counts' data and correctly labels them 'X', 'Y', 'Z'
    by inspecting the gates in each circuit.
    """

    print("--- Step 1: Setting up Experiment ---")

    # 1. Define the target quantum state.
    target_state = Statevector.from_label('+')
    print(f"Target state |+> = {np.round(target_state.data, 3)}")

    # 2. Define simulators
    noiseless_sim = AerSimulator()
    print("Loading FakeVigo backend…")
    fake_backend = FakeVigoV2()
    noise_model = NoiseModel.from_backend(fake_backend)

    # 3. Build a noisy simulator
    noisy_sim = AerSimulator(
        noise_model = noise_model,
        coupling_map = fake_backend.configuration().coupling_map,
        basis_gates  = noise_model.basis_gates
    )
    print("Simulators created.")

    # 3. Setup the tomography experiment
    qst_exp = StateTomography(target_state, measurement_indices=[0])

    shots_per_basis = 1024
    print(f"\nRunning StateTomography for 1-qubit state on 3 bases.")
    print(f"Shots per basis: {shots_per_basis}")

    # --- Step 2: Run the experiments ---
    print("\n--- Step 2: Running Simulations ---")

    print("Running NOISELESS simulation...")
    noiseless_data = qst_exp.run(
        noiseless_sim,
        shots=shots_per_basis,
        seed_simulator=42
    ).block_for_results()

    print("Running NOISY simulation (using FakeVigo)…")
    noisy_data = qst_exp.run(
        noisy_sim,
        shots=shots_per_basis,
        seed_simulator=42
    ).block_for_results()

    print("Simulations complete.")

    # --- Step 3: Analyze tomography results ---
    print("\n--- Step 3: Analyzing Tomography Results (Classical Baseline) ---")

    noiseless_rho = noiseless_data.analysis_results("state").value
    noisy_rho = noisy_data.analysis_results("state").value

    noiseless_fid = state_fidelity(target_state, noiseless_rho)
    noisy_fid     = state_fidelity(target_state, noisy_rho)

    print(f"Target Density Matrix (rho_target):\n{DensityMatrix(target_state).data}")
    print(f"\nNoisy Reconstructed (rho_noisy):\n{np.round(noisy_rho.data, 3)}")

    print("\n--- Baseline Fidelity Scores ---")
    print(f"Noiseless Fidelity: {noiseless_fid:.6f}")
    print(f"Noisy Fidelity:     {noisy_fid:.6f}  <-- THIS IS THE SCORE TO BEAT")


    # --- Step 4: Extract training data for DDM (GATE INSPECTION FIX) ---
    print("\n--- Step 4: Extracting Data for DDM Training ---")

    # The experiment data (results)
    circuit_data = noisy_data.data()
    # The circuits that were run
    experiment_circuits = qst_exp.circuits()
    counts_by_basis = {}

    if len(circuit_data) != len(experiment_circuits):
        print("Error: Mismatch between number of circuits and results.")
    else:
        # Loop through both lists in parallel
        for circuit, result_data in zip(experiment_circuits, circuit_data):
            counts = result_data.get('counts')

            # Get the names of the gates in the circuit
            # We remove 'barrier', 'save_expval', and 'measure' to find the basis gates
            gate_names = set(
                instr.operation.name 
                for instr in circuit.data 
                if instr.operation.name not in ['barrier', 'save_expval', 'measure', 'reset']
            )

            # Now we identify the basis by the gates present
            basis_label = '?'
        
            # Correct check for the measurement basis gates
            if 'PauliMeasX' in gate_names:
                basis_label = 'X'
            elif 'PauliMeasY' in gate_names:
                basis_label = 'Y'
            elif 'PauliMeasZ' in gate_names:
                basis_label = 'Z'
            else:
                print(f"Warning: Could not identify basis from gates: {gate_names}")

            if basis_label != '?':
                counts_by_basis[basis_label] = counts
            else:
                print(f"Warning: Failed to assign basis for circuit {circuit.name}")

    print("\nTraining Data (Counts from Noisy Sim):")
    print(f"Extracted {len(counts_by_basis)} 'counts' objects and sorted by basis.")

    # Ensure all bases are present, even if empty, and print in X, Y, Z order
    all_noisy_counts = []
    for basis in ['X', 'Y', 'Z']:
        # Ensure counts are not None before processing
        counts = counts_by_basis.get(basis)
        if counts is None:
            print(f"Warning: No counts found for basis {basis}. Defaulting to empty.")
            counts = {'0': 0, '1': 0}
        
        all_noisy_counts.append(counts)
        print(f"  Basis {basis}: {counts}")


     # --- Step 5: Visualizing Results ---
    print("\n--- Step 5: Visualizing Results ---")
    
    print("Displaying Bloch Sphere visualizations…")
    
    print("Plotting Target State Bloch Sphere...")
    display(plot_bloch_multivector(target_state, title="Target State (|+>)"))

    print("Plotting Noisy Reconstructed Bloch Sphere...")
    display(plot_bloch_multivector(noisy_rho, title=f"Noisy Reconstruction (Fid: {noisy_fid:.4f})"))

    print("Displaying Density Matrix 'City' plots…")
    
    print("Plotting Target Density Matrix...")
    display(plot_state_city(target_state,
                    title="Target Density Matrix"))
    
    print("Plotting Noisy Reconstructed Density Matrix...")
    display(plot_state_city(noisy_rho,
                    title="Noisy Reconstructed"))

    print("\nAll visualizations complete.")
    print("\nNext step: Copy the 'Basis X/Y/Z' counts into the Step 2 DDM script.")


if __name__ == "__main__":
    main()



In [ ]:
#shots = 4096 case
# --- Installation ---
# Run this once at the top of your notebook.
# You may need to restart the runtime after installation.
!pip install --upgrade --quiet qiskit qiskit-aer qiskit-experiments qiskit-ibm-runtime matplotlib ipython

# --- Python Code ---
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Qiskit Core
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# Qiskit Aer for simulation
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Qiskit Experiments for Tomography
from qiskit_experiments.library import StateTomography

# Fake-backend (for realistic noise model)
from qiskit_ibm_runtime.fake_provider import FakeVigoV2

# Qiskit visualization utilities
from qiskit.visualization import plot_state_city, plot_bloch_multivector

def main():
    """
    Runs a 1-qubit state tomography experiment to establish a baseline.
    Generates noisy 'counts' data and correctly labels them 'X', 'Y', 'Z'
    by inspecting the gates in each circuit.
    """

    print("--- Step 1: Setting up Experiment ---")

    # 1. Define the target quantum state.
    target_state = Statevector.from_label('+')
    print(f"Target state |+> = {np.round(target_state.data, 3)}")

    # 2. Define simulators
    noiseless_sim = AerSimulator()
    print("Loading FakeVigo backend…")
    fake_backend = FakeVigoV2()
    noise_model = NoiseModel.from_backend(fake_backend)

    # 3. Build a noisy simulator
    noisy_sim = AerSimulator(
        noise_model = noise_model,
        coupling_map = fake_backend.configuration().coupling_map,
        basis_gates  = noise_model.basis_gates
    )
    print("Simulators created.")

    # 3. Setup the tomography experiment
    qst_exp = StateTomography(target_state, measurement_indices=[0])

    shots_per_basis = 4096
    print(f"\nRunning StateTomography for 1-qubit state on 3 bases.")
    print(f"Shots per basis: {shots_per_basis}")

    # --- Step 2: Run the experiments ---
    print("\n--- Step 2: Running Simulations ---")

    print("Running NOISELESS simulation...")
    noiseless_data = qst_exp.run(
        noiseless_sim,
        shots=shots_per_basis,
        seed_simulator=42
    ).block_for_results()

    print("Running NOISY simulation (using FakeVigo)…")
    noisy_data = qst_exp.run(
        noisy_sim,
        shots=shots_per_basis,
        seed_simulator=42
    ).block_for_results()

    print("Simulations complete.")

    # --- Step 3: Analyze tomography results ---
    print("\n--- Step 3: Analyzing Tomography Results (Classical Baseline) ---")

    noiseless_rho = noiseless_data.analysis_results("state").value
    noisy_rho = noisy_data.analysis_results("state").value

    noiseless_fid = state_fidelity(target_state, noiseless_rho)
    noisy_fid     = state_fidelity(target_state, noisy_rho)

    print(f"Target Density Matrix (rho_target):\n{DensityMatrix(target_state).data}")
    print(f"\nNoisy Reconstructed (rho_noisy):\n{np.round(noisy_rho.data, 3)}")

    print("\n--- Baseline Fidelity Scores ---")
    print(f"Noiseless Fidelity: {noiseless_fid:.6f}")
    print(f"Noisy Fidelity:     {noisy_fid:.6f}  <-- THIS IS THE SCORE TO BEAT")


    # --- Step 4: Extract training data for DDM (GATE INSPECTION FIX) ---
    print("\n--- Step 4: Extracting Data for DDM Training ---")

    # The experiment data (results)
    circuit_data = noisy_data.data()
    # The circuits that were run
    experiment_circuits = qst_exp.circuits()
    counts_by_basis = {}

    if len(circuit_data) != len(experiment_circuits):
        print("Error: Mismatch between number of circuits and results.")
    else:
        # Loop through both lists in parallel
        for circuit, result_data in zip(experiment_circuits, circuit_data):
            counts = result_data.get('counts')

            # Get the names of the gates in the circuit
            # We remove 'barrier', 'save_expval', and 'measure' to find the basis gates
            gate_names = set(
                instr.operation.name 
                for instr in circuit.data 
                if instr.operation.name not in ['barrier', 'save_expval', 'measure', 'reset']
            )

            # Now we identify the basis by the gates present
            basis_label = '?'
        
            # Correct check for the measurement basis gates
            if 'PauliMeasX' in gate_names:
                basis_label = 'X'
            elif 'PauliMeasY' in gate_names:
                basis_label = 'Y'
            elif 'PauliMeasZ' in gate_names:
                basis_label = 'Z'
            else:
                print(f"Warning: Could not identify basis from gates: {gate_names}")

            if basis_label != '?':
                counts_by_basis[basis_label] = counts
            else:
                print(f"Warning: Failed to assign basis for circuit {circuit.name}")

    print("\nTraining Data (Counts from Noisy Sim):")
    print(f"Extracted {len(counts_by_basis)} 'counts' objects and sorted by basis.")

    # Ensure all bases are present, even if empty, and print in X, Y, Z order
    all_noisy_counts = []
    for basis in ['X', 'Y', 'Z']:
        # Ensure counts are not None before processing
        counts = counts_by_basis.get(basis)
        if counts is None:
            print(f"Warning: No counts found for basis {basis}. Defaulting to empty.")
            counts = {'0': 0, '1': 0}
        
        all_noisy_counts.append(counts)
        print(f"  Basis {basis}: {counts}")


     # --- Step 5: Visualizing Results ---
    print("\n--- Step 5: Visualizing Results ---")
    
    print("Displaying Bloch Sphere visualizations…")
    
    print("Plotting Target State Bloch Sphere...")
    display(plot_bloch_multivector(target_state, title="Target State (|+>)"))

    print("Plotting Noisy Reconstructed Bloch Sphere...")
    display(plot_bloch_multivector(noisy_rho, title=f"Noisy Reconstruction (Fid: {noisy_fid:.4f})"))

    print("Displaying Density Matrix 'City' plots…")
    
    print("Plotting Target Density Matrix...")
    display(plot_state_city(target_state,
                    title="Target Density Matrix"))
    
    print("Plotting Noisy Reconstructed Density Matrix...")
    display(plot_state_city(noisy_rho,
                    title="Noisy Reconstructed"))

    print("\nAll visualizations complete.")
    print("\nNext step: Copy the 'Basis X/Y/Z' counts into the Step 2 DDM script.")


if __name__ == "__main__":
    main()



# Step 2: DDM

In [ ]:

# --- Python Code ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import collections

# --- Step 1: Paste Your Data Here ---
# TODO:
# Copy the *actual* output from your Step 1 script and paste it below.
# The basis order MUST be X, Y, Z.

'''`
Training Data (Counts from Noisy Sim):
Extracted 3 'counts' objects and sorted by basis.
  Basis X: {'0': 940, '1': 84}
  Basis Y: {'0': 545, '1': 479}
  Basis Z: {'1': 497, '0': 527}
'''


counts_X = {'0': 940, '1': 84}  # <-- PASTE YOUR X COUNTS (from Basis ('X',))
counts_Y = {'0': 545, '1': 479}  # <-- PASTE YOUR Y COUNTS (from Basis ('Y',))
counts_Z = {'1': 497, '0': 527} # <-- PASTE YOUR Z COUNTS (from Basis ('Z',))



# --- This script continues from here ---
original_noisy_counts = [counts_X, counts_Y, counts_Z]
BASIS_LABELS = ['X', 'Y', 'Z']

# --- Step 2a: Process Data for ML ---

class BitstringDataset(Dataset):
    """
    Takes a Qiskit Counts dictionary and "unrolls" it into
    a list of individual bitstrings (shots) for training.
    """
    def __init__(self, counts_dict, basis_id):
        self.basis_id = basis_id
        self.data = []

        # Unroll the counts
        for bit_str, num_shots in counts_dict.items():
            # For 1Q, bit_str is '0' or '1'. We convert to int 0 or 1.
            if num_shots > 0:
                bit_val = int(bit_str)
                # Add this bit 'num_shots' times
                self.data.extend([bit_val] * num_shots)
        
        # Convert to a tensor
        self.data = torch.tensor(self.data, dtype=torch.long)
        self.basis_tensor = torch.tensor(basis_id, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return the clean bit and its basis ID
        return self.data[idx], self.basis_tensor

# --- Step 2b: Define the DDM Neural Network ---

class SimpleMLP(nn.Module):
    """
    A simple MLP to predict noise.
    It's *conditional* on the basis_id.
    """
    def __init__(self, num_timesteps=100, num_bases=3):
        super().__init__()
        
        # Embeddings for timestep and basis
        self.time_emb = nn.Embedding(num_timesteps + 1, 32) # +1 for t=0
        self.basis_emb = nn.Embedding(num_bases, 32)
        
        # Input layer: 1 (bit value) + 32 (time) + 32 (basis) = 65
        self.net = nn.Sequential(
            nn.Linear(1 + 32 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 2) # Output: Logits for '0' and '1'
        )

    def forward(self, noisy_x, t, basis_id):
        # x is (batch_size, 1)
        # t is (batch_size,)
        # basis_id is (batch_size,)
        
        # Get embeddings
        t_emb = self.time_emb(t)
        b_emb = self.basis_emb(basis_id)
        
        # Reshape x from (batch_size,) to (batch_size, 1) and cast to float
        noisy_x = noisy_x.float().view(-1, 1)

        # Concatenate inputs
        x_in = torch.cat([noisy_x, t_emb, b_emb], dim=1)
        
        # Return logits (raw scores for each class)
        return self.net(x_in)

# --- Step 2c & 3: Define the Diffusion Model ---

class BitstringDDM:
    """
    Manages the DDM training (Step 2) and sampling (Step 3) process
    for categorical (binary) data.
    """
    def __init__(self, model, num_timesteps=100, device='cuda'):
        self.model = model.to(device)
        self.num_timesteps = num_timesteps
        self.device = device

        # Define the noise schedule (transition matrices)
        p_stay = torch.linspace(1.0, 0.5, num_timesteps + 1)
        
        self.Q = torch.zeros(num_timesteps + 1, 2, 2) # T, to_state, from_state
        
        for t in range(1, num_timesteps + 1):
            p = p_stay[t]
            # Q[t, 0, 0] = P(0 | 0) = p
            # Q[t, 1, 0] = P(1 | 0) = 1-p
            # Q[t, 0, 1] = P(0 | 1) = 1-p
            # Q[t, 1, 1] = P(1 | 1) = p
            self.Q[t] = torch.tensor([[p, 1-p], [1-p, p]])

        # *** BUGFIX: *** Move Q to the device (e.g., GPU)
        self.Q = self.Q.to(device)

    def forward_diffusion(self, x_0, t):
        """Adds t steps of noise to a clean bit x_0."""
        q_t = self.Q[t] # Get the (2,2) transition matrix for timestep t
        
        # *** BUGFIX: ***
        # 'q_t' is shape (2, 2).
        # 'x_0' is shape (batch_size,).
        # We need to select the correct column from q_t for each item in the batch.
        # The result 'probs' should be (batch_size, 2).
        probs = q_t[:, x_0] # This selects columns, but 't' is also a batch.
        
        # The 't' tensor is shape (batch_size,).
        # 'x_0' is shape (batch_size,).
        # We need to select Q[t_i] for each item i, then select the column x_0[i].
        
        # 1. Get the transition matrix for each 't' in the batch
        # Q_t_batch shape: (batch_size, 2, 2)
        Q_t_batch = self.Q[t]
        
        # 2. We need to select the column indexed by x_0 for each item in the batch.
        # This is a bit tricky. We can use gather.
        # Or, since it's just 2x2:
        # P(x_t | x_0=0) is Q_t_batch[:, :, 0]
        # P(x_t | x_0=1) is Q_t_batch[:, :, 1]
        
        # Create a view of x_0 for indexing
        # x_0_idx shape: (batch_size, 1, 1)
        x_0_idx = x_0.view(-1, 1, 1)
        
        # 'probs' will be (batch_size, 2)
        # We select column 0 or 1 from the (batch_size, 2, 2) tensor
        # We expand x_0_idx to (batch_size, 2, 1) to select the column
        probs = Q_t_batch.gather(dim=2, index=x_0_idx.expand(-1, 2, -1)).squeeze(2)

        # Sample x_t from the categorical distribution
        x_t = torch.multinomial(probs, num_samples=1).squeeze(1)
        return x_t.to(self.device)

    def train_step(self, x_0, basis_id):
        """Performs one training step."""
        batch_size = x_0.shape[0]
        x_0 = x_0.to(self.device)
        basis_id = basis_id.to(self.device)
        
        # 1. Sample a random timestep t
        t = torch.randint(1, self.num_timesteps + 1, (batch_size,), device=self.device)

        # 2. Create noisy x_t
        x_t = self.forward_diffusion(x_0, t)

        # 3. Get model's prediction of the *clean* state (x_0)
        pred_x_0_logits = self.model(x_t, t, basis_id)

        # 4. Calculate the loss (Cross-entropy between predicted clean and actual clean)
        loss = F.cross_entropy(pred_x_0_logits, x_0)
        return loss

    def sample(self, num_samples, basis_id):
        """(STEP 3) Generate new, denoised samples."""
        print(f"\n--- Step 3: Sampling {num_samples} bits for Basis {BASIS_LABELS[basis_id]} ---")
        
        # 1. Start with pure noise (random bits) at time T
        x_t = torch.randint(0, 2, (num_samples,), device=self.device)
        
        b_id_tensor = torch.full_like(x_t, fill_value=basis_id)
        
        # 2. Iteratively denoise from T down to 1
        for t in reversed(range(1, self.num_timesteps + 1)):
            with torch.no_grad():
                # Get model's prediction of the clean state
                t_tensor = torch.full_like(x_t, fill_value=t)
                pred_x_0_logits = self.model(x_t, t_tensor, b_id_tensor)
                
                # Convert logits to probabilities (softmax)
                pred_x_0_probs = F.softmax(pred_x_0_logits, dim=1)
                
                # Ancestral sampling step:
                # 1. Sample the predicted clean state
                x_0_pred = torch.multinomial(pred_x_0_probs, 1).squeeze(1)
                
                if t > 1:
                    # 2. Add t-1 steps of noise to it to get x_{t-1}
                    t_prev_tensor = torch.full_like(x_t, fill_value=t-1)
                    x_t = self.forward_diffusion(x_0_pred, t_prev_tensor)
                else:
                    # Last step, the final sample is our prediction
                    x_t = x_0_pred

        # 3. Return the final denoised samples
        return x_t.cpu().numpy()

# --- Main Execution ---

def run_ddm_poc():
    # --- Setup ---
    NUM_EPOCHS = 200
    BATCH_SIZE = 512
    LEARNING_RATE = 1e-3
    NUM_TIMESTEPS = 100
    
    # Use GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Step 2a (Data) ---
    print("\n--- Step 2a: Processing Data ---")
    print(f"Using Original Noisy Data:")
    print(f"  X: {counts_X}")
    print(f"  Y: {counts_Y}")
    print(f"  Z: {counts_Z}")

    dataset_x = BitstringDataset(counts_X, basis_id=0)
    dataset_y = BitstringDataset(counts_Y, basis_id=1)
    dataset_z = BitstringDataset(counts_Z, basis_id=2)
    
    # Check for empty datasets
    if not dataset_x or not dataset_y or not dataset_z:
        print("\n*** ERROR: One or more datasets are empty. ***")
        print("Please re-run Step 1 and paste the correct non-empty 'counts' dictionaries.")
        return

    # Combine all 3 datasets into one
    full_dataset = ConcatDataset([dataset_x, dataset_y, dataset_z])
    dataloader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # We will sample the same number of shots we trained on
    total_shots_per_basis = {
        0: len(dataset_x),
        1: len(dataset_y),
        2: len(dataset_z)
    }
    
    print(f"Total training samples (shots): {len(full_dataset)}")
    print(f"Shots per basis (for sampling): {total_shots_per_basis}")


    # --- Step 2b (Model) ---
    model = SimpleMLP(num_timesteps=NUM_TIMESTEPS, num_bases=3)
    
    # --- Step 2c (Training) ---
    print("\n--- Step 2c: Training DDM ---")
    
    ddm = BitstringDDM(model, num_timesteps=NUM_TIMESTEPS, device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for x_0, basis_id in dataloader:
            optimizer.zero_grad()
            loss = ddm.train_step(x_0, basis_id)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.6f}")

    print("--- Training Complete (Step 2 Done) ---")

    # --- Step 3: Sampling ---
    generated_counts = []
    
    for basis_id in range(3):
        # We generate the same number of samples we trained on
        num_samples = total_shots_per_basis[basis_id]
        samples = ddm.sample(num_samples=num_samples, basis_id=basis_id)
        
        # Count the bits to create a new Counts dictionary
        # Use collections.Counter to correctly handle cases where one bit is 0
        counts_counter = collections.Counter(samples)
        counts = {'0': counts_counter.get(0, 0), '1': counts_counter.get(1, 0)}
        
        generated_counts.append(counts)

    # --- Results ---
    print("\n--- Comparison: Original vs. DDM Denoised ---")
    for i in range(3):
        label = BASIS_LABELS[i]
        print(f"\nBasis {label}:")
        print(f"  Original Noisy: {original_noisy_counts[i]}")
        print(f"  DDM Denoised:   {generated_counts[i]}")

    print("\nNext step (Step 4): Reconstruct a density matrix from the 'DDM Denoised' counts.")

# Run the full Proof-of-Concept
if __name__ == "__main__":
    run_ddm_poc()



In [ ]:
#4096 shots
# --- Python Code ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import collections

# --- Step 1: Paste Your Data Here ---
# TODO:
# Copy the *actual* output from your Step 1 script and paste it below.
# The basis order MUST be X, Y, Z.

'''`

Training Data (Counts from Noisy Sim):
Extracted 3 'counts' objects and sorted by basis.
  Basis X: {'0': 3785, '1': 311}
  Basis Y: {'0': 2152, '1': 1944}
  Basis Z: {'1': 2067, '0': 2029}

'''


counts_X = {'0': 3785, '1': 311}  # <-- PASTE YOUR X COUNTS (from Basis ('X',))
counts_Y = {'0': 2152, '1': 1944}  # <-- PASTE YOUR Y COUNTS (from Basis ('Y',))
counts_Z = {'1': 2067, '0': 2029}  # <-- PASTE YOUR Z COUNTS (from Basis ('Z',))



# --- This script continues from here ---
original_noisy_counts = [counts_X, counts_Y, counts_Z]
BASIS_LABELS = ['X', 'Y', 'Z']

# --- Step 2a: Process Data for ML ---

class BitstringDataset(Dataset):
    """
    Takes a Qiskit Counts dictionary and "unrolls" it into
    a list of individual bitstrings (shots) for training.
    """
    def __init__(self, counts_dict, basis_id):
        self.basis_id = basis_id
        self.data = []

        # Unroll the counts
        for bit_str, num_shots in counts_dict.items():
            # For 1Q, bit_str is '0' or '1'. We convert to int 0 or 1.
            if num_shots > 0:
                bit_val = int(bit_str)
                # Add this bit 'num_shots' times
                self.data.extend([bit_val] * num_shots)
        
        # Convert to a tensor
        self.data = torch.tensor(self.data, dtype=torch.long)
        self.basis_tensor = torch.tensor(basis_id, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return the clean bit and its basis ID
        return self.data[idx], self.basis_tensor

# --- Step 2b: Define the DDM Neural Network ---

class SimpleMLP(nn.Module):
    """
    A simple MLP to predict noise.
    It's *conditional* on the basis_id.
    """
    def __init__(self, num_timesteps=100, num_bases=3):
        super().__init__()
        
        # Embeddings for timestep and basis
        self.time_emb = nn.Embedding(num_timesteps + 1, 32) # +1 for t=0
        self.basis_emb = nn.Embedding(num_bases, 32)
        
        # Input layer: 1 (bit value) + 32 (time) + 32 (basis) = 65
        self.net = nn.Sequential(
            nn.Linear(1 + 32 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 2) # Output: Logits for '0' and '1'
        )

    def forward(self, noisy_x, t, basis_id):
        # x is (batch_size, 1)
        # t is (batch_size,)
        # basis_id is (batch_size,)
        
        # Get embeddings
        t_emb = self.time_emb(t)
        b_emb = self.basis_emb(basis_id)
        
        # Reshape x from (batch_size,) to (batch_size, 1) and cast to float
        noisy_x = noisy_x.float().view(-1, 1)

        # Concatenate inputs
        x_in = torch.cat([noisy_x, t_emb, b_emb], dim=1)
        
        # Return logits (raw scores for each class)
        return self.net(x_in)

# --- Step 2c & 3: Define the Diffusion Model ---

class BitstringDDM:
    """
    Manages the DDM training (Step 2) and sampling (Step 3) process
    for categorical (binary) data.
    """
    def __init__(self, model, num_timesteps=100, device='cuda'):
        self.model = model.to(device)
        self.num_timesteps = num_timesteps
        self.device = device

        # Define the noise schedule (transition matrices)
        p_stay = torch.linspace(1.0, 0.5, num_timesteps + 1)
        
        self.Q = torch.zeros(num_timesteps + 1, 2, 2) # T, to_state, from_state
        
        for t in range(1, num_timesteps + 1):
            p = p_stay[t]
            # Q[t, 0, 0] = P(0 | 0) = p
            # Q[t, 1, 0] = P(1 | 0) = 1-p
            # Q[t, 0, 1] = P(0 | 1) = 1-p
            # Q[t, 1, 1] = P(1 | 1) = p
            self.Q[t] = torch.tensor([[p, 1-p], [1-p, p]])

        # *** BUGFIX: *** Move Q to the device (e.g., GPU)
        self.Q = self.Q.to(device)

    def forward_diffusion(self, x_0, t):
        """Adds t steps of noise to a clean bit x_0."""
        q_t = self.Q[t] # Get the (2,2) transition matrix for timestep t
        
        # *** BUGFIX: ***
        # 'q_t' is shape (2, 2).
        # 'x_0' is shape (batch_size,).
        # We need to select the correct column from q_t for each item in the batch.
        # The result 'probs' should be (batch_size, 2).
        probs = q_t[:, x_0] # This selects columns, but 't' is also a batch.
        
        # The 't' tensor is shape (batch_size,).
        # 'x_0' is shape (batch_size,).
        # We need to select Q[t_i] for each item i, then select the column x_0[i].
        
        # 1. Get the transition matrix for each 't' in the batch
        # Q_t_batch shape: (batch_size, 2, 2)
        Q_t_batch = self.Q[t]
        
        # 2. We need to select the column indexed by x_0 for each item in the batch.
        # This is a bit tricky. We can use gather.
        # Or, since it's just 2x2:
        # P(x_t | x_0=0) is Q_t_batch[:, :, 0]
        # P(x_t | x_0=1) is Q_t_batch[:, :, 1]
        
        # Create a view of x_0 for indexing
        # x_0_idx shape: (batch_size, 1, 1)
        x_0_idx = x_0.view(-1, 1, 1)
        
        # 'probs' will be (batch_size, 2)
        # We select column 0 or 1 from the (batch_size, 2, 2) tensor
        # We expand x_0_idx to (batch_size, 2, 1) to select the column
        probs = Q_t_batch.gather(dim=2, index=x_0_idx.expand(-1, 2, -1)).squeeze(2)

        # Sample x_t from the categorical distribution
        x_t = torch.multinomial(probs, num_samples=1).squeeze(1)
        return x_t.to(self.device)

    def train_step(self, x_0, basis_id):
        """Performs one training step."""
        batch_size = x_0.shape[0]
        x_0 = x_0.to(self.device)
        basis_id = basis_id.to(self.device)
        
        # 1. Sample a random timestep t
        t = torch.randint(1, self.num_timesteps + 1, (batch_size,), device=self.device)

        # 2. Create noisy x_t
        x_t = self.forward_diffusion(x_0, t)

        # 3. Get model's prediction of the *clean* state (x_0)
        pred_x_0_logits = self.model(x_t, t, basis_id)

        # 4. Calculate the loss (Cross-entropy between predicted clean and actual clean)
        loss = F.cross_entropy(pred_x_0_logits, x_0)
        return loss

    def sample(self, num_samples, basis_id):
        """(STEP 3) Generate new, denoised samples."""
        print(f"\n--- Step 3: Sampling {num_samples} bits for Basis {BASIS_LABELS[basis_id]} ---")
        
        # 1. Start with pure noise (random bits) at time T
        x_t = torch.randint(0, 2, (num_samples,), device=self.device)
        
        b_id_tensor = torch.full_like(x_t, fill_value=basis_id)
        
        # 2. Iteratively denoise from T down to 1
        for t in reversed(range(1, self.num_timesteps + 1)):
            with torch.no_grad():
                # Get model's prediction of the clean state
                t_tensor = torch.full_like(x_t, fill_value=t)
                pred_x_0_logits = self.model(x_t, t_tensor, b_id_tensor)
                
                # Convert logits to probabilities (softmax)
                pred_x_0_probs = F.softmax(pred_x_0_logits, dim=1)
                
                # Ancestral sampling step:
                # 1. Sample the predicted clean state
                x_0_pred = torch.multinomial(pred_x_0_probs, 1).squeeze(1)
                
                if t > 1:
                    # 2. Add t-1 steps of noise to it to get x_{t-1}
                    t_prev_tensor = torch.full_like(x_t, fill_value=t-1)
                    x_t = self.forward_diffusion(x_0_pred, t_prev_tensor)
                else:
                    # Last step, the final sample is our prediction
                    x_t = x_0_pred

        # 3. Return the final denoised samples
        return x_t.cpu().numpy()

# --- Main Execution ---

def run_ddm_poc():
    # --- Setup ---
    NUM_EPOCHS = 200
    BATCH_SIZE = 512
    LEARNING_RATE = 1e-3
    NUM_TIMESTEPS = 100
    
    # Use GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Step 2a (Data) ---
    print("\n--- Step 2a: Processing Data ---")
    print(f"Using Original Noisy Data:")
    print(f"  X: {counts_X}")
    print(f"  Y: {counts_Y}")
    print(f"  Z: {counts_Z}")

    dataset_x = BitstringDataset(counts_X, basis_id=0)
    dataset_y = BitstringDataset(counts_Y, basis_id=1)
    dataset_z = BitstringDataset(counts_Z, basis_id=2)
    
    # Check for empty datasets
    if not dataset_x or not dataset_y or not dataset_z:
        print("\n*** ERROR: One or more datasets are empty. ***")
        print("Please re-run Step 1 and paste the correct non-empty 'counts' dictionaries.")
        return

    # Combine all 3 datasets into one
    full_dataset = ConcatDataset([dataset_x, dataset_y, dataset_z])
    dataloader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # We will sample the same number of shots we trained on
    total_shots_per_basis = {
        0: len(dataset_x),
        1: len(dataset_y),
        2: len(dataset_z)
    }
    
    print(f"Total training samples (shots): {len(full_dataset)}")
    print(f"Shots per basis (for sampling): {total_shots_per_basis}")


    # --- Step 2b (Model) ---
    model = SimpleMLP(num_timesteps=NUM_TIMESTEPS, num_bases=3)
    
    # --- Step 2c (Training) ---
    print("\n--- Step 2c: Training DDM ---")
    
    ddm = BitstringDDM(model, num_timesteps=NUM_TIMESTEPS, device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for x_0, basis_id in dataloader:
            optimizer.zero_grad()
            loss = ddm.train_step(x_0, basis_id)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.6f}")

    print("--- Training Complete (Step 2 Done) ---")

    # --- Step 3: Sampling ---
    generated_counts = []
    
    for basis_id in range(3):
        # We generate the same number of samples we trained on
        num_samples = total_shots_per_basis[basis_id]
        samples = ddm.sample(num_samples=num_samples, basis_id=basis_id)
        
        # Count the bits to create a new Counts dictionary
        # Use collections.Counter to correctly handle cases where one bit is 0
        counts_counter = collections.Counter(samples)
        counts = {'0': counts_counter.get(0, 0), '1': counts_counter.get(1, 0)}
        
        generated_counts.append(counts)

    # --- Results ---
    print("\n--- Comparison: Original vs. DDM Denoised ---")
    for i in range(3):
        label = BASIS_LABELS[i]
        print(f"\nBasis {label}:")
        print(f"  Original Noisy: {original_noisy_counts[i]}")
        print(f"  DDM Denoised:   {generated_counts[i]}")

    print("\nNext step (Step 4): Reconstruct a density matrix from the 'DDM Denoised' counts.")

# Run the full Proof-of-Concept
if __name__ == "__main__":
    run_ddm_poc()



# Step 3: Reconstruction and FID

In [ ]:


# --- Python Code ---
import numpy as np
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# --- Pauli Matrices ---
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=complex)
SIGMA_Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=complex)
IDENTITY = np.array([[1, 0], [0, 1]], dtype=complex)

# === START: TODO ===

'''

Basis X:
  Original Noisy: {'0': 940, '1': 84}
  DDM Denoised:   {'0': 931, '1': 93}

Basis Y:
  Original Noisy: {'0': 545, '1': 479}
  DDM Denoised:   {'0': 564, '1': 460}

Basis Z:
  Original Noisy: {'1': 497, '0': 527}
  DDM Denoised:   {'0': 445, '1': 579}

'''

# TODO 1:
# Copy the 'DDM Denoised' counts from your Step 2/3 script output.
ddm_counts_X = {'0': 931, '1': 93}   # <-- PASTE YOUR DDM 'X' COUNTS
ddm_counts_Y = {'0': 564, '1': 460} # <-- PASTE YOUR DDM 'Y' COUNTS
ddm_counts_Z = {'0': 445, '1': 579}  # <-- PASTE YOUR DDM 'Z' COUNTS

# TODO 2:
# Copy the 'Noisy Fidelity' (the score to beat) from your Step 1 output.
CLASSICAL_BASELINE_FIDELITY = 0.917969  # <-- PASTE YOUR BASELINE FIDELITY

# === END: TODO ===


def calculate_expectation_value(counts):
    """
    Calculates the expectation value <Z> from a Qiskit counts dictionary.
    
    <Z> = P(0) - P(1)
    """
    total_shots = sum(counts.values())
    if total_shots == 0:
        return 0.0 # Avoid division by zero
        
    p_0 = counts.get('0', 0) / total_shots
    p_1 = counts.get('1', 0) / total_shots
    
    return p_0 - p_1

def reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z):
    """
    Reconstructs the 1-qubit density matrix rho using the formula:
    rho = (1/2) * (I + <X>sigma_X + <Y>sigma_Y + <Z>sigma_Z)
    """
    rho = 0.5 * (IDENTITY + 
                   exp_val_X * SIGMA_X + 
                   exp_val_Y * SIGMA_Y + 
                   exp_val_Z * SIGMA_Z)
    
    # Return as a Qiskit DensityMatrix object for fidelity calculation
    return DensityMatrix(rho)

def run_reconstruction():
    print("--- Step 4: Reconstructing DDM's Density Matrix ---")

    # 1. Define our target state again for fidelity calculation
    target_state = Statevector.from_label('+')

    # 2. Calculate expectation values from the DDM's counts
    exp_val_X = calculate_expectation_value(ddm_counts_X)
    exp_val_Y = calculate_expectation_value(ddm_counts_Y)
    exp_val_Z = calculate_expectation_value(ddm_counts_Z)

    print(f"DDM Expectation Values:")
    print(f"  <X> = {exp_val_X:.6f}")
    print(f"  <Y> = {exp_val_Y:.6f}")
    print(f"  <Z> = {exp_val_Z:.6f}")

    # 3. Reconstruct the density matrix
    rho_ddm = reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z)
    
    print(f"\nDDM Reconstructed (rho_ddm):\n{np.round(rho_ddm.data, 3)}")

    # 4. Calculate the final fidelity
    ddm_fidelity = state_fidelity(target_state, rho_ddm)

    print("\n--- Step 5: Final Results & Comparison ---")
    print(f"  Classical Baseline Fidelity: {CLASSICAL_BASELINE_FIDELITY:.6f}")
    print(f"  DDM Denoised Fidelity:       {ddm_fidelity:.6f}")

    # 5. Print the verdict
    if ddm_fidelity > CLASSICAL_BASELINE_FIDELITY:
        improvement = ddm_fidelity - CLASSICAL_BASELINE_FIDELITY
        print(f"\n*** SUCCESS! ***")
        print(f"The DDM improved the fidelity by {improvement:.6f}.")
    else:
        print(f"\n--- No Improvement ---")
        print("The DDM fidelity did not beat the classical baseline.")
        print("This may be due to a short training run or unlucky sampling.")
        print("Try re-running Step 2/3, or increasing NUM_EPOCHS.")

if __name__ == "__main__":
    run_reconstruction()



In [ ]:
# for 4096 shots

# --- Python Code ---
import numpy as np
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# --- Pauli Matrices ---
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=complex)
SIGMA_Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=complex)
IDENTITY = np.array([[1, 0], [0, 1]], dtype=complex)

# === START: TODO ===

'''
Basis X:
  Original Noisy: {'0': 3785, '1': 311}
  DDM Denoised:   {'0': 3602, '1': 494}

Basis Y:
  Original Noisy: {'0': 2152, '1': 1944}
  DDM Denoised:   {'0': 2068, '1': 2028}

Basis Z:
  Original Noisy: {'1': 2067, '0': 2029}
  DDM Denoised:   {'0': 1741, '1': 2355}

'''

# TODO 1:
# Copy the 'DDM Denoised' counts from your Step 2/3 script output.
ddm_counts_X = {'0': 3602, '1': 494}   # <-- PASTE YOUR DDM 'X' COUNTS
ddm_counts_Y = {'0': 2068, '1': 2028}  # <-- PASTE YOUR DDM 'Y' COUNTS
ddm_counts_Z = {'0': 1741, '1': 2355}  # <-- PASTE YOUR DDM 'Z' COUNTS

# TODO 2:
# Copy the 'Noisy Fidelity' (the score to beat) from your Step 1 output.
CLASSICAL_BASELINE_FIDELITY = 0.924072  # <-- PASTE YOUR BASELINE FIDELITY

# === END: TODO ===


def calculate_expectation_value(counts):
    """
    Calculates the expectation value <Z> from a Qiskit counts dictionary.
    
    <Z> = P(0) - P(1)
    """
    total_shots = sum(counts.values())
    if total_shots == 0:
        return 0.0 # Avoid division by zero
        
    p_0 = counts.get('0', 0) / total_shots
    p_1 = counts.get('1', 0) / total_shots
    
    return p_0 - p_1

def reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z):
    """
    Reconstructs the 1-qubit density matrix rho using the formula:
    rho = (1/2) * (I + <X>sigma_X + <Y>sigma_Y + <Z>sigma_Z)
    """
    rho = 0.5 * (IDENTITY + 
                   exp_val_X * SIGMA_X + 
                   exp_val_Y * SIGMA_Y + 
                   exp_val_Z * SIGMA_Z)
    
    # Return as a Qiskit DensityMatrix object for fidelity calculation
    return DensityMatrix(rho)

def run_reconstruction():
    print("--- Step 4: Reconstructing DDM's Density Matrix ---")

    # 1. Define our target state again for fidelity calculation
    target_state = Statevector.from_label('+')

    # 2. Calculate expectation values from the DDM's counts
    exp_val_X = calculate_expectation_value(ddm_counts_X)
    exp_val_Y = calculate_expectation_value(ddm_counts_Y)
    exp_val_Z = calculate_expectation_value(ddm_counts_Z)

    print(f"DDM Expectation Values:")
    print(f"  <X> = {exp_val_X:.6f}")
    print(f"  <Y> = {exp_val_Y:.6f}")
    print(f"  <Z> = {exp_val_Z:.6f}")

    # 3. Reconstruct the density matrix
    rho_ddm = reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z)
    
    print(f"\nDDM Reconstructed (rho_ddm):\n{np.round(rho_ddm.data, 3)}")

    # 4. Calculate the final fidelity
    ddm_fidelity = state_fidelity(target_state, rho_ddm)

    print("\n--- Step 5: Final Results & Comparison ---")
    print(f"  Classical Baseline Fidelity: {CLASSICAL_BASELINE_FIDELITY:.6f}")
    print(f"  DDM Denoised Fidelity:       {ddm_fidelity:.6f}")

    # 5. Print the verdict
    if ddm_fidelity > CLASSICAL_BASELINE_FIDELITY:
        improvement = ddm_fidelity - CLASSICAL_BASELINE_FIDELITY
        print(f"\n*** SUCCESS! ***")
        print(f"The DDM improved the fidelity by {improvement:.6f}.")
    else:
        print(f"\n--- No Improvement ---")
        print("The DDM fidelity did not beat the classical baseline.")
        print("This may be due to a short training run or unlucky sampling.")
        print("Try re-running Step 2/3, or increasing NUM_EPOCHS.")

if __name__ == "__main__":
    run_reconstruction()



# Second iteration of steps 2 and 3 using a better model

In [ ]:
# Step 2 with more powerful model

# --- Python Code ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import collections

# --- Step 1: Paste Your Data Here ---
# TODO:
# Copy the *actual* output from your Step 1 script and paste it below.
# The basis order MUST be X, Y, Z.

'''
  Basis X: {'0': 940, '1': 84}
  Basis Y: {'0': 545, '1': 479}
  Basis Z: {'1': 497, '0': 527}
'''

counts_X = {'0': 940, '1': 84}  # <-- PASTE YOUR X COUNTS
counts_Y = {'0': 545, '1': 479}  # <-- PASTE YOUR Y COUNTS
counts_Z = {'0': 497, '1': 527}  # <-- PASTE YOUR Z COUNTS


# --- This script continues from here ---
original_noisy_counts = [counts_X, counts_Y, counts_Z]
BASIS_LABELS = ['X', 'Y', 'Z']

# --- Step 2a: Process Data for ML ---

class BitstringDataset(Dataset):
    """
    Takes a Qiskit Counts dictionary and "unrolls" it into
    a list of individual bitstrings (shots) for training.
    """
    def __init__(self, counts_dict, basis_id):
        self.basis_id = basis_id
        self.data = []

        # Unroll the counts
        for bit_str, num_shots in counts_dict.items():
            if num_shots > 0:
                bit_val = int(bit_str)
                self.data.extend([bit_val] * num_shots)
        
        self.data = torch.tensor(self.data, dtype=torch.long)
        self.basis_tensor = torch.tensor(basis_id, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.basis_tensor

# --- Step 2b: Define the (UPGRADED) DDM Neural Network ---

class UpgradedMLP(nn.Module):
    """
    A more powerful MLP to predict noise.
    It's *conditional* on the basis_id.
    
    *** CHANGES: ***
    - Increased embedding dimensions (32 -> 128)
    - Deeper and wider network (128 -> 256)
    - Added an extra layer
    """
    def __init__(self, num_timesteps=100, num_bases=3):
        super().__init__()
        
        EMBED_DIM = 128  # Was 32
        
        self.time_emb = nn.Embedding(num_timesteps + 1, EMBED_DIM) # +1 for t=0
        self.basis_emb = nn.Embedding(num_bases, EMBED_DIM)
        
        # Input layer: 1 (bit) + 128 (time) + 128 (basis) = 257
        HIDDEN_DIM = 256 # Was 128
        
        self.net = nn.Sequential(
            nn.Linear(1 + EMBED_DIM + EMBED_DIM, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, HIDDEN_DIM), # New layer
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, HIDDEN_DIM), # Was 128
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 2) # Output: Logits for '0' and '1'
        )

    def forward(self, noisy_x, t, basis_id):
        t_emb = self.time_emb(t)
        b_emb = self.basis_emb(basis_id)
        noisy_x = noisy_x.float().view(-1, 1)
        x_in = torch.cat([noisy_x, t_emb, b_emb], dim=1)
        return self.net(x_in)

# --- Step 2c & 3: Define the Diffusion Model ---

class BitstringDDM:
    """
    Manages the DDM training (Step 2) and sampling (Step 3) process.
    (This class is unchanged, but will use the new UpgradedMLP)
    """
    def __init__(self, model, num_timesteps=100, device='cuda'):
        self.model = model.to(device)
        self.num_timesteps = num_timesteps
        self.device = device

        # Define the noise schedule (transition matrices)
        p_stay = torch.linspace(1.0, 0.5, num_timesteps + 1)
        
        self.Q = torch.zeros(num_timesteps + 1, 2, 2) # T, to_state, from_state
        
        for t in range(1, num_timesteps + 1):
            p = p_stay[t]
            self.Q[t] = torch.tensor([[p, 1-p], [1-p, p]])

        self.Q = self.Q.to(device)

    def forward_diffusion(self, x_0, t):
        """Adds t steps of noise to a clean bit x_0."""
        Q_t_batch = self.Q[t]
        x_0_idx = x_0.view(-1, 1, 1)
        probs = Q_t_batch.gather(dim=2, index=x_0_idx.expand(-1, 2, -1)).squeeze(2)
        x_t = torch.multinomial(probs, num_samples=1).squeeze(1)
        return x_t.to(self.device)

    def train_step(self, x_0, basis_id):
        """Performs one training step."""
        batch_size = x_0.shape[0]
        x_0 = x_0.to(self.device)
        basis_id = basis_id.to(self.device)
        
        t = torch.randint(1, self.num_timesteps + 1, (batch_size,), device=self.device)
        x_t = self.forward_diffusion(x_0, t)
        pred_x_0_logits = self.model(x_t, t, basis_id)
        loss = F.cross_entropy(pred_x_0_logits, x_0)
        return loss

    def sample(self, num_samples, basis_id):
        """(STEP 3) Generate new, denoised samples."""
        print(f"\n--- Step 3: Sampling {num_samples} bits for Basis {BASIS_LABELS[basis_id]} ---")
        
        x_t = torch.randint(0, 2, (num_samples,), device=self.device)
        b_id_tensor = torch.full_like(x_t, fill_value=basis_id)
        
        for t in reversed(range(1, self.num_timesteps + 1)):
            with torch.no_grad():
                t_tensor = torch.full_like(x_t, fill_value=t)
                pred_x_0_logits = self.model(x_t, t_tensor, b_id_tensor)
                pred_x_0_probs = F.softmax(pred_x_0_logits, dim=1)
                x_0_pred = torch.multinomial(pred_x_0_probs, 1).squeeze(1)
                
                if t > 1:
                    t_prev_tensor = torch.full_like(x_t, fill_value=t-1)
                    x_t = self.forward_diffusion(x_0_pred, t_prev_tensor)
                else:
                    x_t = x_0_pred

        return x_t.cpu().numpy()

# --- Main Execution ---

def run_ddm_poc():
    # --- Setup ---
    # *** CHANGES: ***
    # - NUM_EPOCHS = 300 (Keep it high)
    # - BATCH_SIZE = 128 (Smaller for more updates)
    # - LEARNING_RATE = 3e-4 (Slightly lower for stability)
    NUM_EPOCHS = 300
    BATCH_SIZE = 128
    LEARNING_RATE = 3e-4 
    NUM_TIMESTEPS = 100
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Step 2a (Data) ---
    print("\n--- Step 2a: Processing Data ---")
    print(f"Using Original Noisy Data:")
    print(f"  X: {counts_X}")
    print(f"  Y: {counts_Y}")
    print(f"  Z: {counts_Z}")

    dataset_x = BitstringDataset(counts_X, basis_id=0)
    dataset_y = BitstringDataset(counts_Y, basis_id=1)
    dataset_z = BitstringDataset(counts_Z, basis_id=2)
    
    if len(dataset_x) == 0 or len(dataset_y) == 0 or len(dataset_z) == 0:
        print("\n*** ERROR: One or more datasets are empty. ***")
        print("Please re-run Step 1 and paste the correct non-empty 'counts' dictionaries.")
        return

    full_dataset = ConcatDataset([dataset_x, dataset_y, dataset_z])
    dataloader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    total_shots_per_basis = {
        0: len(dataset_x),
        1: len(dataset_y),
        2: len(dataset_z)
    }
    
    print(f"Total training samples (shots): {len(full_dataset)}")
    print(f"Shots per basis (for sampling): {total_shots_per_basis}")


    # --- Step 2b (Model) ---
    # *** CHANGE: Using new, more powerful model ***
    model = UpgradedMLP(num_timesteps=NUM_TIMESTEPS, num_bases=3)
    
    # --- Step 2c (Training) ---
    print("\n--- Step 2c: Training DDM (Upgraded Model) ---")
    
    ddm = BitstringDDM(model, num_timesteps=NUM_TIMESTEPS, device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for x_0, basis_id in dataloader:
            optimizer.zero_grad()
            loss = ddm.train_step(x_0, basis_id)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 25 == 0: # Print every 25 epochs
            avg_loss = total_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.6f}")

    print("--- Training Complete (Step 2 Done) ---")

    # --- Step 3: Sampling ---
    generated_counts = []
    
    for basis_id in range(3):
        num_samples = total_shots_per_basis[basis_id]
        samples = ddm.sample(num_samples=num_samples, basis_id=basis_id)
        
        counts_counter = collections.Counter(samples)
        counts = {'0': counts_counter.get(0, 0), '1': counts_counter.get(1, 0)}
        
        generated_counts.append(counts)

    # --- Results ---
    print("\n--- Comparison: Original vs. DDM Denoised ---")
    for i in range(3):
        label = BASIS_LABELS[i]
        print(f"\nBasis {label}:")
        print(f"  Original Noisy: {original_noisy_counts[i]}")
        print(f"  DDM Denoised:   {generated_counts[i]}")

    print("\nNext step (Step 4): Reconstruct a density matrix from the 'DDM Denoised' counts.")

if __name__ == "__main__":
    run_ddm_poc()


In [ ]:


# --- Python Code ---
import numpy as np
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# --- Pauli Matrices ---
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=complex)
SIGMA_Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=complex)
IDENTITY = np.array([[1, 0], [0, 1]], dtype=complex)

# === START: TODO ===

'''


Basis X:
  Original Noisy: {'0': 940, '1': 84}
  DDM Denoised:   {'0': 952, '1': 72}

Basis Y:
  Original Noisy: {'0': 545, '1': 479}
  DDM Denoised:   {'0': 525, '1': 499}

Basis Z:
  Original Noisy: {'0': 497, '1': 527}
  DDM Denoised:   {'0': 454, '1': 570}

'''

# TODO 1:
# Copy the 'DDM Denoised' counts from your Step 2/3 script output.
ddm_counts_X = {'0': 952, '1': 84}   # <-- PASTE YOUR DDM 'X' COUNTS
ddm_counts_Y = {'0': 525, '1': 499} # <-- PASTE YOUR DDM 'Y' COUNTS
ddm_counts_Z = {'0': 454, '1': 570}  # <-- PASTE YOUR DDM 'Z' COUNTS

# TODO 2:
# Copy the 'Noisy Fidelity' (the score to beat) from your Step 1 output.
CLASSICAL_BASELINE_FIDELITY = 0.917969  # <-- PASTE YOUR BASELINE FIDELITY

# === END: TODO ===


def calculate_expectation_value(counts):
    """
    Calculates the expectation value <Z> from a Qiskit counts dictionary.
    
    <Z> = P(0) - P(1)
    """
    total_shots = sum(counts.values())
    if total_shots == 0:
        return 0.0 # Avoid division by zero
        
    p_0 = counts.get('0', 0) / total_shots
    p_1 = counts.get('1', 0) / total_shots
    
    return p_0 - p_1

def reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z):
    """
    Reconstructs the 1-qubit density matrix rho using the formula:
    rho = (1/2) * (I + <X>sigma_X + <Y>sigma_Y + <Z>sigma_Z)
    """
    rho = 0.5 * (IDENTITY + 
                   exp_val_X * SIGMA_X + 
                   exp_val_Y * SIGMA_Y + 
                   exp_val_Z * SIGMA_Z)
    
    # Return as a Qiskit DensityMatrix object for fidelity calculation
    return DensityMatrix(rho)

def run_reconstruction():
    print("--- Step 4: Reconstructing DDM's Density Matrix ---")

    # 1. Define our target state again for fidelity calculation
    target_state = Statevector.from_label('+')

    # 2. Calculate expectation values from the DDM's counts
    exp_val_X = calculate_expectation_value(ddm_counts_X)
    exp_val_Y = calculate_expectation_value(ddm_counts_Y)
    exp_val_Z = calculate_expectation_value(ddm_counts_Z)

    print(f"DDM Expectation Values:")
    print(f"  <X> = {exp_val_X:.6f}")
    print(f"  <Y> = {exp_val_Y:.6f}")
    print(f"  <Z> = {exp_val_Z:.6f}")

    # 3. Reconstruct the density matrix
    rho_ddm = reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z)
    
    print(f"\nDDM Reconstructed (rho_ddm):\n{np.round(rho_ddm.data, 3)}")

    # 4. Calculate the final fidelity
    ddm_fidelity = state_fidelity(target_state, rho_ddm)

    print("\n--- Step 5: Final Results & Comparison ---")
    print(f"  Classical Baseline Fidelity: {CLASSICAL_BASELINE_FIDELITY:.6f}")
    print(f"  DDM Denoised Fidelity:       {ddm_fidelity:.6f}")

    # 5. Print the verdict
    if ddm_fidelity > CLASSICAL_BASELINE_FIDELITY:
        improvement = ddm_fidelity - CLASSICAL_BASELINE_FIDELITY
        print(f"\n*** SUCCESS! ***")
        print(f"The DDM improved the fidelity by {improvement:.6f}.")
    else:
        print(f"\n--- No Improvement ---")
        print("The DDM fidelity did not beat the classical baseline.")
        print("This may be due to a short training run or unlucky sampling.")
        print("Try re-running Step 2/3, or increasing NUM_EPOCHS.")

if __name__ == "__main__":
    run_reconstruction()



In [ ]:
#4096
# Step 2 with more powerful model

# --- Python Code ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import collections

# --- Step 1: Paste Your Data Here ---
# TODO:
# Copy the *actual* output from your Step 1 script and paste it below.
# The basis order MUST be X, Y, Z.

'''
  Basis X: {'0': 3785, '1': 311}
  Basis Y: {'0': 2152, '1': 1944}
  Basis Z: {'1': 2067, '0': 2029}
'''

counts_X = {'0': 3785, '1': 311}  # <-- PASTE YOUR X COUNTS
counts_Y = {'0': 2152, '1': 1944}  # <-- PASTE YOUR Y COUNTS
counts_Z = {'0': 2067, '1': 2029}  # <-- PASTE YOUR Z COUNTS


# --- This script continues from here ---
original_noisy_counts = [counts_X, counts_Y, counts_Z]
BASIS_LABELS = ['X', 'Y', 'Z']

# --- Step 2a: Process Data for ML ---

class BitstringDataset(Dataset):
    """
    Takes a Qiskit Counts dictionary and "unrolls" it into
    a list of individual bitstrings (shots) for training.
    """
    def __init__(self, counts_dict, basis_id):
        self.basis_id = basis_id
        self.data = []

        # Unroll the counts
        for bit_str, num_shots in counts_dict.items():
            if num_shots > 0:
                bit_val = int(bit_str)
                self.data.extend([bit_val] * num_shots)
        
        self.data = torch.tensor(self.data, dtype=torch.long)
        self.basis_tensor = torch.tensor(basis_id, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.basis_tensor

# --- Step 2b: Define the (UPGRADED) DDM Neural Network ---

class UpgradedMLP(nn.Module):
    """
    A more powerful MLP to predict noise.
    It's *conditional* on the basis_id.
    
    *** CHANGES: ***
    - Increased embedding dimensions (32 -> 128)
    - Deeper and wider network (128 -> 256)
    - Added an extra layer
    """
    def __init__(self, num_timesteps=100, num_bases=3):
        super().__init__()
        
        EMBED_DIM = 128  # Was 32
        
        self.time_emb = nn.Embedding(num_timesteps + 1, EMBED_DIM) # +1 for t=0
        self.basis_emb = nn.Embedding(num_bases, EMBED_DIM)
        
        # Input layer: 1 (bit) + 128 (time) + 128 (basis) = 257
        HIDDEN_DIM = 256 # Was 128
        
        self.net = nn.Sequential(
            nn.Linear(1 + EMBED_DIM + EMBED_DIM, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, HIDDEN_DIM), # New layer
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, HIDDEN_DIM), # Was 128
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 2) # Output: Logits for '0' and '1'
        )

    def forward(self, noisy_x, t, basis_id):
        t_emb = self.time_emb(t)
        b_emb = self.basis_emb(basis_id)
        noisy_x = noisy_x.float().view(-1, 1)
        x_in = torch.cat([noisy_x, t_emb, b_emb], dim=1)
        return self.net(x_in)

# --- Step 2c & 3: Define the Diffusion Model ---

class BitstringDDM:
    """
    Manages the DDM training (Step 2) and sampling (Step 3) process.
    (This class is unchanged, but will use the new UpgradedMLP)
    """
    def __init__(self, model, num_timesteps=100, device='cuda'):
        self.model = model.to(device)
        self.num_timesteps = num_timesteps
        self.device = device

        # Define the noise schedule (transition matrices)
        p_stay = torch.linspace(1.0, 0.5, num_timesteps + 1)
        
        self.Q = torch.zeros(num_timesteps + 1, 2, 2) # T, to_state, from_state
        
        for t in range(1, num_timesteps + 1):
            p = p_stay[t]
            self.Q[t] = torch.tensor([[p, 1-p], [1-p, p]])

        self.Q = self.Q.to(device)

    def forward_diffusion(self, x_0, t):
        """Adds t steps of noise to a clean bit x_0."""
        Q_t_batch = self.Q[t]
        x_0_idx = x_0.view(-1, 1, 1)
        probs = Q_t_batch.gather(dim=2, index=x_0_idx.expand(-1, 2, -1)).squeeze(2)
        x_t = torch.multinomial(probs, num_samples=1).squeeze(1)
        return x_t.to(self.device)

    def train_step(self, x_0, basis_id):
        """Performs one training step."""
        batch_size = x_0.shape[0]
        x_0 = x_0.to(self.device)
        basis_id = basis_id.to(self.device)
        
        t = torch.randint(1, self.num_timesteps + 1, (batch_size,), device=self.device)
        x_t = self.forward_diffusion(x_0, t)
        pred_x_0_logits = self.model(x_t, t, basis_id)
        loss = F.cross_entropy(pred_x_0_logits, x_0)
        return loss

    def sample(self, num_samples, basis_id):
        """(STEP 3) Generate new, denoised samples."""
        print(f"\n--- Step 3: Sampling {num_samples} bits for Basis {BASIS_LABELS[basis_id]} ---")
        
        x_t = torch.randint(0, 2, (num_samples,), device=self.device)
        b_id_tensor = torch.full_like(x_t, fill_value=basis_id)
        
        for t in reversed(range(1, self.num_timesteps + 1)):
            with torch.no_grad():
                t_tensor = torch.full_like(x_t, fill_value=t)
                pred_x_0_logits = self.model(x_t, t_tensor, b_id_tensor)
                pred_x_0_probs = F.softmax(pred_x_0_logits, dim=1)
                x_0_pred = torch.multinomial(pred_x_0_probs, 1).squeeze(1)
                
                if t > 1:
                    t_prev_tensor = torch.full_like(x_t, fill_value=t-1)
                    x_t = self.forward_diffusion(x_0_pred, t_prev_tensor)
                else:
                    x_t = x_0_pred

        return x_t.cpu().numpy()

# --- Main Execution ---

def run_ddm_poc():
    # --- Setup ---
    # *** CHANGES: ***
    # - NUM_EPOCHS = 300 (Keep it high)
    # - BATCH_SIZE = 128 (Smaller for more updates)
    # - LEARNING_RATE = 3e-4 (Slightly lower for stability)
    NUM_EPOCHS = 300
    BATCH_SIZE = 128
    LEARNING_RATE = 3e-4 
    NUM_TIMESTEPS = 100
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Step 2a (Data) ---
    print("\n--- Step 2a: Processing Data ---")
    print(f"Using Original Noisy Data:")
    print(f"  X: {counts_X}")
    print(f"  Y: {counts_Y}")
    print(f"  Z: {counts_Z}")

    dataset_x = BitstringDataset(counts_X, basis_id=0)
    dataset_y = BitstringDataset(counts_Y, basis_id=1)
    dataset_z = BitstringDataset(counts_Z, basis_id=2)
    
    if len(dataset_x) == 0 or len(dataset_y) == 0 or len(dataset_z) == 0:
        print("\n*** ERROR: One or more datasets are empty. ***")
        print("Please re-run Step 1 and paste the correct non-empty 'counts' dictionaries.")
        return

    full_dataset = ConcatDataset([dataset_x, dataset_y, dataset_z])
    dataloader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    total_shots_per_basis = {
        0: len(dataset_x),
        1: len(dataset_y),
        2: len(dataset_z)
    }
    
    print(f"Total training samples (shots): {len(full_dataset)}")
    print(f"Shots per basis (for sampling): {total_shots_per_basis}")


    # --- Step 2b (Model) ---
    # *** CHANGE: Using new, more powerful model ***
    model = UpgradedMLP(num_timesteps=NUM_TIMESTEPS, num_bases=3)
    
    # --- Step 2c (Training) ---
    print("\n--- Step 2c: Training DDM (Upgraded Model) ---")
    
    ddm = BitstringDDM(model, num_timesteps=NUM_TIMESTEPS, device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for x_0, basis_id in dataloader:
            optimizer.zero_grad()
            loss = ddm.train_step(x_0, basis_id)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 25 == 0: # Print every 25 epochs
            avg_loss = total_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.6f}")

    print("--- Training Complete (Step 2 Done) ---")

    # --- Step 3: Sampling ---
    generated_counts = []
    
    for basis_id in range(3):
        num_samples = total_shots_per_basis[basis_id]
        samples = ddm.sample(num_samples=num_samples, basis_id=basis_id)
        
        counts_counter = collections.Counter(samples)
        counts = {'0': counts_counter.get(0, 0), '1': counts_counter.get(1, 0)}
        
        generated_counts.append(counts)

    # --- Results ---
    print("\n--- Comparison: Original vs. DDM Denoised ---")
    for i in range(3):
        label = BASIS_LABELS[i]
        print(f"\nBasis {label}:")
        print(f"  Original Noisy: {original_noisy_counts[i]}")
        print(f"  DDM Denoised:   {generated_counts[i]}")

    print("\nNext step (Step 4): Reconstruct a density matrix from the 'DDM Denoised' counts.")

if __name__ == "__main__":
    run_ddm_poc()


In [ ]:
# for 4096 shots

# --- Python Code ---
import numpy as np
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix

# --- Pauli Matrices ---
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=complex)
SIGMA_Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=complex)
IDENTITY = np.array([[1, 0], [0, 1]], dtype=complex)

# === START: TODO ===

'''
Basis X:
  Original Noisy: {'0': 3785, '1': 311}
  DDM Denoised:   {'0': 3857, '1': 239}

Basis Y:
  Original Noisy: {'0': 2152, '1': 1944}
  DDM Denoised:   {'0': 2178, '1': 1918}

Basis Z:
  Original Noisy: {'0': 2067, '1': 2029}
  DDM Denoised:   {'0': 2111, '1': 1985}


'''

# TODO 1:
# Copy the 'DDM Denoised' counts from your Step 2/3 script output.
ddm_counts_X = {'0': 3857, '1': 239}   # <-- PASTE YOUR DDM 'X' COUNTS
ddm_counts_Y = {'0': 2178, '1': 1918}  # <-- PASTE YOUR DDM 'Y' COUNTS
ddm_counts_Z = {'0': 2111, '1': 1985}  # <-- PASTE YOUR DDM 'Z' COUNTS

# TODO 2:
# Copy the 'Noisy Fidelity' (the score to beat) from your Step 1 output.
CLASSICAL_BASELINE_FIDELITY = 0.924072  # <-- PASTE YOUR BASELINE FIDELITY

# === END: TODO ===


def calculate_expectation_value(counts):
    """
    Calculates the expectation value <Z> from a Qiskit counts dictionary.
    
    <Z> = P(0) - P(1)
    """
    total_shots = sum(counts.values())
    if total_shots == 0:
        return 0.0 # Avoid division by zero
        
    p_0 = counts.get('0', 0) / total_shots
    p_1 = counts.get('1', 0) / total_shots
    
    return p_0 - p_1

def reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z):
    """
    Reconstructs the 1-qubit density matrix rho using the formula:
    rho = (1/2) * (I + <X>sigma_X + <Y>sigma_Y + <Z>sigma_Z)
    """
    rho = 0.5 * (IDENTITY + 
                   exp_val_X * SIGMA_X + 
                   exp_val_Y * SIGMA_Y + 
                   exp_val_Z * SIGMA_Z)
    
    # Return as a Qiskit DensityMatrix object for fidelity calculation
    return DensityMatrix(rho)

def run_reconstruction():
    print("--- Step 4: Reconstructing DDM's Density Matrix ---")

    # 1. Define our target state again for fidelity calculation
    target_state = Statevector.from_label('+')

    # 2. Calculate expectation values from the DDM's counts
    exp_val_X = calculate_expectation_value(ddm_counts_X)
    exp_val_Y = calculate_expectation_value(ddm_counts_Y)
    exp_val_Z = calculate_expectation_value(ddm_counts_Z)

    print(f"DDM Expectation Values:")
    print(f"  <X> = {exp_val_X:.6f}")
    print(f"  <Y> = {exp_val_Y:.6f}")
    print(f"  <Z> = {exp_val_Z:.6f}")

    # 3. Reconstruct the density matrix
    rho_ddm = reconstruct_rho(exp_val_X, exp_val_Y, exp_val_Z)
    
    print(f"\nDDM Reconstructed (rho_ddm):\n{np.round(rho_ddm.data, 3)}")

    # 4. Calculate the final fidelity
    ddm_fidelity = state_fidelity(target_state, rho_ddm)

    print("\n--- Step 5: Final Results & Comparison ---")
    print(f"  Classical Baseline Fidelity: {CLASSICAL_BASELINE_FIDELITY:.6f}")
    print(f"  DDM Denoised Fidelity:       {ddm_fidelity:.6f}")

    # 5. Print the verdict
    if ddm_fidelity > CLASSICAL_BASELINE_FIDELITY:
        improvement = ddm_fidelity - CLASSICAL_BASELINE_FIDELITY
        print(f"\n*** SUCCESS! ***")
        print(f"The DDM improved the fidelity by {improvement:.6f}.")
    else:
        print(f"\n--- No Improvement ---")
        print("The DDM fidelity did not beat the classical baseline.")
        print("This may be due to a short training run or unlucky sampling.")
        print("Try re-running Step 2/3, or increasing NUM_EPOCHS.")

if __name__ == "__main__":
    run_reconstruction()



# The Big Picture: Our Goal

Our goal is to see if a  Denoising Diffusion Model is better at "cleaning" noisy quantum data than the standard classical methods.

In other words, a contest: **"Classical Tomography" vs. "DDM Tomography."**

---

## Step 1: The "Experiment" and "Setting the Score"




1.  **Prepare the State:** We define a perfect 1-qubit target state, `|+>`. This is our "ground truth."
2.  **Run the "Classical Race":** We run Qiskit's built-in `StateTomography` on a *noisy* simulator (`FakeVigo`). This simulator adds errors, just like real hardware.
3.  **Get the Classical Score:** This classical method analyzes the noisy data and produces `rho_noisy`, which has a fidelity of **`0.917969`**. This is the **baseline score to beat**.
4.  **Get the Training Data:** The *same* noisy simulation gives us the `Counts` (the raw X, Y, Z measurement results). This noisy, messy data is the "training data" we will feed to our AI.



---

### Step 2: Training the AI (The "Brain")

This is where we build and train our "brain" (the model used for prediction).

1.  **Load Data:** We "unroll" the `Noisy Counts`. For example, `{'X': {'0': 940, '1': 84}}` becomes a giant list containing 940 `0`s and 84 `1`s, where each bit is "tagged" with the `X-Basis` label.
2.  **The "Training Game":** We play a game with our AI for 300 epochs:
    * **Corrupt:** We take a *clean* bit from our dataset (e.g., `0`), and add a *random* amount of noise, flipping it (e.g., to `1`). This is the **Forward Diffusion** process.
    * **Ask:** We show the model the noisy bit `1` and ask, "I corrupted this bit at noise level `t=50` and it's from the `X-Basis`. What was the original?"
    * **Guess:** The model makes a prediction (e.g., `0`).
    * **Learn:** We tell the AI, "You were correct!" (or "You were wrong!"), and it updates its neural network.



After 300 epochs, our AI is incredibly good at this "denoising" game. It has learned the hidden rules, like "If the basis is X, the clean bit should almost *always* be `0`."

---

### Step 3: Running the AI (The "Generator")

Now we use our "smart" AI to generate *brand new, clean data*. We run the training game **in reverse**.

1.  **Start with Garbage:** We create 1024 *completely random* bits (pure noise). This is `x_T` (time `T=100`).
2.  **Denoise Step-by-Step:** We ask our AI, "Here is a batch of garbage `x_100` for the `X-Basis`. What's one step cleaner (`x_99`)?"
3.  **Iterate:** The AI gives us `x_99`. We feed *that* back to the AI and ask for `x_98`. We repeat this 100 times, "walking" the data from pure noise back to a clean state. This is the **Reverse Diffusion** process.
4.  **Get Clean Data:** Our final result, `x_0`, is a batch of 1024 *newly generated* bits that represent the AI's "idea" of a perfect, clean X-Basis measurement.
5.  **Output:** We do this for all three bases (X, Y, Z) and get our `DDM Denoised` counts.



---

### Step 4: Building the "AI's State"


We takes the AI's *clean* counts and builds a quantum state from them.

1.  **Get Clean Counts:** We paste in the `DDM Denoised` counts from Step 3.
2.  **Calculate Expectation Values:** We convert the clean counts into three numbers:
    * `DDM Denoised X {'0': 1020, '1': 4}` $\rightarrow$ $\langle X \rangle = (1020 - 4) / 1024 = \mathbf{0.992}$
    * ...and so on for $\langle Y \rangle$ and $\langle Z \rangle$.
3.  **Build the State:** We use the physics formula $\rho = \frac{1}{2}(I + \langle X \rangle\sigma_X + \langle Y \rangle\sigma_Y + \langle Z \rangle\sigma_Z)$ to build the final `rho_ddm` (our AI's $2 \times 2$ density matrix).

---

### Step 5: The "Judgment Day"

This is the end of the race. We compare the scores.

1.  **Calculate DDM Fidelity:** We compare our AI's state (`rho_ddm`) to the perfect "ground truth" state (`|+>`) and get our final score: `DDM Denoised Fidelity`.
2.  **Compare the Scores:** The script prints the final comparison:
    * `Classical Baseline Fidelity: 0.917969`
    * `DDM Denoised Fidelity:   0.909180` 



That is the entire, end-to-end research pipeline, for now

## Next steps
1. Documentation
        Need to document every thing used and decision made. This is provide option for introducing varioations. 
4. Metrics-> Runtime, fair comparison between both approaches, Finding ddm advantage
        Advantage of DL based methods established in this paper. 
6. how to consider the basis for multile qubit
7. Custom noise exploration
       based on changing probabbilities of any particular basis (X,Y,Z) error occuring.
5. classical method vs DL method
        with LI method
7. basis of measurement, shots considered
       selective/ random basis considered to compare
       what basis to try to use
